In [1]:
import classes
from amadeus import Client, ResponseError
from hotels1 import hotelFunctions
import amadeus
#import api
import hotels1
import hotelTests
import transport
from transport import transportFunctions

In [2]:
amadeus = Client(
        client_id='CGwOmHn7cmfAIuUcbqUiaPC5LAyAvwAG',
        client_secret='rKvILHDsjxcCh6yq'
    )
hotel = hotelFunctions(amadeus)

In [6]:
hotels = hotel.getHotelsByGeocode(32.079664, 34.767410)

In [7]:
for i in hotels:
    print(i)

{'Duration': None, 'Cost': 153.0, 'Times:': ['2022-06-16', '2022-06-17'], 'Title': 'Best Western Regency Suites', 'Dest': '32.079664,34.76741'}
{'Duration': None, 'Cost': 413.1, 'Times:': ['2022-06-16', '2022-06-17'], 'Title': 'Sheraton Tel Aviv', 'Dest': '32.079664,34.76741'}


In [15]:
transport = transportFunctions()
transports = transport.getTransport(32.0845191,34.8037962,32.0791345,34.7924341)


In [16]:
for i in transports:
    print(i)

{'Duration': datetime.timedelta(seconds=480), 'Cost': 0, 'Times:': [datetime.datetime(2022, 6, 16, 17, 28), datetime.datetime(2022, 6, 16, 17, 36)], 'Title': 'pedestrian', 'Dest': '32.082614,34.800225'}
{'Duration': datetime.timedelta(seconds=180), 'Cost': None, 'Times:': [datetime.datetime(2022, 6, 16, 17, 36), datetime.datetime(2022, 6, 16, 17, 39)], 'Title': 'bus', 'Dest': '32.077723,34.790819'}
{'Duration': datetime.timedelta(seconds=300), 'Cost': 0, 'Times:': [datetime.datetime(2022, 6, 16, 17, 39), datetime.datetime(2022, 6, 16, 17, 44)], 'Title': 'pedestrian', 'Dest': '32.079135,34.792434'}


In [3]:
attractions = hotel.getActivities(32.079664, 34.767410, 2)

In [4]:
for i in attractions:
    print(i)

{'Duration': None, 'Cost': 25.0, 'Times:': [None, None], 'Title': 'Tel Aviv Pub Crawl'}
{'Duration': None, 'Cost': 96.0, 'Times:': [None, None], 'Title': 'From Jerusalem: Masada and Dead Sea Tour'}
{'Duration': None, 'Cost': 34.0, 'Times:': [None, None], 'Title': 'Tel Aviv: Street Food Tour'}
{'Duration': None, 'Cost': 80.0, 'Times:': [None, None], 'Title': 'Jerusalem Day Tour from Tel Aviv'}
{'Duration': None, 'Cost': 65.0, 'Times:': [None, None], 'Title': 'Tel Aviv: Jerusalem Old City Late Morning Tour'}
{'Duration': None, 'Cost': 53.0, 'Times:': [None, None], 'Title': 'Tel Aviv: Private Group Guided Bike Tour'}
